## 1. train spacy ner model
## 2. use spacy ner model recognize skill word
## 3. use model output building bigger train_dataset
## 4. use the bigger train_dataset train ner model

In [1]:
import pandas as pd
from collections import Counter
import spacy,re,json
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from spacy import displacy  

from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

import re
from pprint import pprint
import os
import warnings
warnings.filterwarnings('ignore')

from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding

# load dataset

In [3]:
PATH ='data/file_ner.json1'

annotated_data = []
lines=[]
with open(PATH, 'r',encoding = 'utf-8 sig') as f:
    lines = f.readlines()

for line in lines:
    data = json.loads(line)
    text = data['text']
    entities = []
    for label in data['labels']:
        entities.append(label)
    annotated_data.append((text, {"entities": data['labels']}))

In [4]:
training_data=  annotated_data[0:200]
test_data = annotated_data[200:220]

In [5]:
print(len(training_data), len(test_data))

200 20


In [6]:
training_data[0]

('"Abhishek Jha Application Development Associate - Accenture  Bengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a  • To work for an organization which provides me the opportunity to improve my skills and knowledge for my individual and company\'s growth in best possible ways.  Willing to relocate to: Bangalore, Karnataka  WORK EXPERIENCE  Application Development Associate  Accenture -  November 2017 to Present  Role: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries for the Bot which will be triggered based on given input. Also, Training the bot for different possible utterances (Both positive and negative), which will be given as input by the user.  EDUCATION  B.E in Information science and engineering  B.v.b college of engineering and technology -  Hubli, Karnataka  August 2013 to June 2017  12th in Mathematics  Woodbine modern school  April 2011 to March 2013  10th  Kendriya Vidyalaya  April 2001 to March 2011  SKILLS  C (

# training spacy ner model

In [6]:
def main(model=None, output_dir=None, n_iter=150):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in training_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(training_data)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(training_data, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)

    # test the trained model
    for text, _ in training_data:
        doc = nlp(text)
#         print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
#         print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text, _ in training_data:
            doc = nlp2(text)
#             print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
#             print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

# trained this model with 100 iterations in google colab using GPU runtime with loss: Losses {'ner': 465.24571515528373} 
main(output_dir='ner_model/') 

Created blank 'en' model
Losses {'ner': 17567.603133779616}
Losses {'ner': 2903.422977228077}
Losses {'ner': 2710.5855747221613}
Losses {'ner': 2708.4187101242155}
Losses {'ner': 2506.7801823824993}
Losses {'ner': 2512.528394833455}
Losses {'ner': 2224.032057501509}
Losses {'ner': 2147.6532758825106}
Losses {'ner': 2121.262002958075}
Losses {'ner': 2056.781032268246}
Losses {'ner': 1871.2554673784186}
Losses {'ner': 1973.6627182794016}
Losses {'ner': 1723.5610437213365}
Losses {'ner': 1751.2862675389479}
Losses {'ner': 1704.2621516120648}
Losses {'ner': 1688.9362768871267}
Losses {'ner': 1636.237727484327}
Losses {'ner': 1496.232257265009}
Losses {'ner': 1387.8809005422925}
Losses {'ner': 1489.9176070718079}
Losses {'ner': 1259.005263842997}
Losses {'ner': 1276.5537040650902}
Losses {'ner': 1466.8408615950034}
Losses {'ner': 1292.2291972040523}
Losses {'ner': 1339.42745977291}
Losses {'ner': 1231.438941460793}
Losses {'ner': 1264.6791204632661}
Losses {'ner': 1325.7836225470344}
Losses

# use ner model recoginze skill word

In [3]:
ner_model = spacy.load('ner_model/')

In [119]:
skill_set = {}
link_set = {}

In [ ]:
ind = 0

for root,dirs,files in os.walk(r"E:\dataset\dataset"):
    for file in files:
        ind += 1
        
        try:
            my_file = open( os.path.join(root,file),encoding = 'utf-8' )
            my_string = my_file.readlines()

            for string in my_string[1:]:
                doc = ner_model( string )

                temp_list = []
                for chunk in doc.ents:
                    temp_list.append( chunk.text )
                    
                temp_set = set( temp_list )
                
                for tem in list(temp_set):
                    if tem not in skill_set.keys():
                        print( tem )
                        
                    if skill_set.get( tem, 0 ) == 0:
                        skill_set[tem] = 1
                    else:
                        skill_set[tem] += 1
                        
                for t1 in list(temp_set):
                    for t2 in list(temp_set):
                        if t1.text != t2.text:
                            if link_set.get( t1+'|'+t2, 0 ) == 0:
                                link_set[t1+'|'+t2] = 1
                            else:
                                link_set[t1+'|'+t2] += 1
                            

            my_file.close()
        
            if ind % 100 == 0:
                print( '------------------------------------------------------------------------',ind )
                
#             if ind > 2000:
#                 break
                
        except:
            pass
        


02/05/2019

BlackRock

SQL

Opportunity

R

Solaris
Java
Python
C++
Spark
R
C
SQL
Map/Reduce
Excel
software prototyping
Data Pre
Deep Learning
engineering
Machine Learning
PHP
RESPONSIBILITIES

Required

Consumer experience
TV planning
Programmatic buying
Practical experience
Artificial Intelligence
TensorFlow
Hadoop
Data Scientist
Data Analytics
Data Science
Deep Neural Networks
experience

Responsibilities

·
machine learning
SAP HANA
sunny.patel@disys.com      
Business Intelligence
Docker
Google Platform
AWS
; Testing
Informatica
MATLAB
Product Support
S3
hire
recruitment
Redshift
Athena
SPSS
SAS
Tensorflow
Matlab
THE CHALLENGE
Oracle
Data analysis experience
NoSQL
non-parametric
Perl
C#
create insightful
Linux
MySQL
Data Management
Unit testing
Python

Data Scientist

Apache
ETL
Predictive Analytics
Mobile Identity
Advanced SQL

Influence

etc 

SUMMARY

problem-solving
predictive modeling
Data Mining
Spark ?
Cloud Computing
Data Visualization
MapReduce
MS Reporting
Tableau
Elasti

# get Skill word

In [2]:
PATH ='word/words.txt'

word_set = set()
lines=[]

ind = 0
with open(PATH, 'r') as f:
    lines = f.readlines()
    
    for line in  lines:
        word_set.add( str(line).strip().lower() )
        
        ind+=1
        if ind % 10 == 0:
            print(  str(line).strip()  )
                
word_set

computer science
Qliksense
visualization
Artificial Intelligence
Docker
Matlab
Data Management
Data Mining
Python
deep learning
Map Reduce
SQL Server
Deep experience
Azure
Convolutional Neural Networks
SQL
Statistics
BigQuery
EC2
Map reduce
Machine Learning
Pig
Python
PERL
OpenCV
Linux
RDBMS
team player
NumPy
Pig
Microsoft office
Data Operations Engineer
Oracle Consulting
Unlimited Vacation
GitHub
Personalization
Construct
Office
non-linear
elasticsearch
Linux scripting
TSCSM
ElasticSearch
Data Scientist
Advanced SQL Server
Tableau
Business Intelligence
hypothesis testing
Spark
Required
regression analysis
Design reports
MatLab
Cognitive Sciences
PyCharm
TopBraid
Colorado
Feature Engineering
Experience querying
project planning
Spotfire
Mac
Data Analytics
SENIOR
traditional risk management
strategic partnerships
Oracle
Unit
operational performance
Customer Service
Unix Shell Scripting
Statistical analysis
Data visualization experience
Deep experience solving
Grafana
POCs
SSRS
Unlimited

{'.net',
 '.net programmer',
 'a/b testing',
 'abilities',
 'access',
 'accountabilities',
 'acl',
 'actively shares resources;',
 'adobe analytics',
 'advance sql',
 'advanced analysis',
 'advanced analytics',
 'advanced data analytics',
 'advanced degree',
 'advanced excel',
 'advanced experience',
 'advanced google analytics',
 'advanced manufacturing',
 'advanced marketing',
 'advanced ms',
 'advanced programmatic expertise',
 'advanced python',
 'advanced quantitative',
 'advanced sas',
 'advanced sql server',
 'advanced standard',
 'advanced statistical analysis',
 'agile methodology',
 'ai',
 'ai platforms',
 'airflow',
 'alelo',
 'algorithmic development',
 'alpine data labs',
 'alta it services',
 'amazon aurora',
 'amazon redshift',
 'amicable',
 'aml',
 'analysis',
 'analytics',
 'analytics experience',
 'analyticsea',
 'andreessen horowitz',
 'angular js',
 'anomaly detection',
 'apache',
 'apache gradle',
 'apache hadoop',
 'apache kafka',
 'apache pig',
 'apache rya',
 'a

In [ ]:
# 

In [3]:
nlp = spacy.load("en_core_web_sm")

In [4]:
total_list = []
word_list = []

In [5]:
rootdir = 'JD_dataset'
file_list = os.listdir(rootdir) #列出文件夹下所有的目录与文件

for i in range( 0,len(file_list) ):
    if i % 100 == 0:
        print(i)
    
    path = os.path.join(rootdir,file_list[i])
#     if os.path.isfile(path):
#     print( path )
    
    if os.path.isfile(path):
        with open(path, 'r',encoding = 'utf-8 sig') as f:
            strings = f.readlines()
            
            for string in strings[1:]:
                for sentence in nlp( string ).sents:
                    
                    if 'www' in sentence.text or '.com' in sentence.text:
                        continue
                        
#                     print( '=' * 120)
#                     if len(sentence.text) > 3:
#                     print( sentence.text )
                    doc = nlp( sentence.text )
            
                    Flag = False
                    words = []
                    
                    number = 0
                    for chunk in doc.noun_chunks:
                        number += 1
                        if str(chunk).lower() in word_set:
                            Flag = True
                            words.append( chunk )
#                         print( chunk,Flag,end ='' )
                        
#                     if number == len( words ) and Flag:
                    if Flag:
                        total_list.append( [ sentence.text.strip() ] )
                        word_list.append( words )
                        
#                     print()
#                     print( '_' * 120)
        

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


## sentence

In [6]:
total_list[:10]

[['The Enterprise Data Scientist I will be a valuable part of the Data Science team who provide statistical analysis, modeling, and vis ualization services for all lines of business and all shared service functions at Farmers Insurance Group.'],
 ['Extract data from a variety of sources (databases, web, text files) and in a variety of formats (structured, unstructured, JSON, Parquet, etc.) and prepare the data for analysis Describe data using qualitative and quantitative aggregation/summarization and build tools (static reports, interactive dashboards, web applications) that allow clients to easily view and understand important patterns and trends in the data Provide quantitative comparisons of data points using advanced statistical analysis including data reduction techniques (PCA, cluste ring), resampling (permutation tests, Monte Carlo testing) as well as traditional statistical hypothesis testing Construct and validate predictive models using a variety of techniques'],
 ['Proficien

##  skill  keyword

In [7]:
word_list[:10]

[[statistical analysis, modeling],
 [databases, web, analysis, advanced statistical analysis, Construct],
 [Tableau, neural networks],
 [Python, SQL, Logistic Regression],
 [Hive],
 [Apache Spark, Scala],
 [Java],
 [databases],
 [computer science, mathematics, statistics, economics, engineering],
 [SQL, data analysis]]

# build dataset

## output test

In [32]:
total_sentence_list = []
total_index_list = []

In [33]:
for index in range( 0, len(total_list[:100]) ):
    try:
        tuplp_list = []

        txt = total_list[index][0]
        doc = nlp(txt)

        temp_sentence = [str(token) for token in doc]
        print( temp_sentence )
#         print( word_list[index] )

        my_word_list = []
        for w in word_list[index]:
            my_word_list.append( [str(token) for token in nlp(str(w))] )

        print( my_word_list )


        total_sentence_list.append( temp_sentence )
        my_ind = 0
        for l in my_word_list:
            temp_Flag = True
            for t in l:
                print( str(t) ,'',end = '') 
                my_ind = temp_sentence.index( str(t), my_ind )
                print( my_ind ,'',end = '') 

                label = ''
                if temp_Flag:
                    label= 'B'
                    print( label ,'|',end = '') 
                    temp_Flag = False
                else:
                    label= 'I'
                    print( label ,'|',end = '') 
                tuplp_list.append( ( label, my_ind ) )

                my_ind += 1
            print()

        total_index_list.append( tuplp_list )
        
    except:
        print( 'Error--------------------------------------------------------------------' )

['The', 'Enterprise', 'Data', 'Scientist', 'I', 'will', 'be', 'a', 'valuable', 'part', 'of', 'the', 'Data', 'Science', 'team', 'who', 'provide', 'statistical', 'analysis', ',', 'modeling', ',', 'and', 'vis', 'ualization', 'services', 'for', 'all', 'lines', 'of', 'business', 'and', 'all', 'shared', 'service', 'functions', 'at', 'Farmers', 'Insurance', 'Group', '.']
[['statistical', 'analysis'], ['modeling']]
statistical 17 B |analysis 18 I |
modeling 20 B |
['Extract', 'data', 'from', 'a', 'variety', 'of', 'sources', '(', 'databases', ',', 'web', ',', 'text', 'files', ')', 'and', 'in', 'a', 'variety', 'of', 'formats', '(', 'structured', ',', 'unstructured', ',', 'JSON', ',', 'Parquet', ',', 'etc', '.', ')', 'and', 'prepare', 'the', 'data', 'for', 'analysis', 'Describe', 'data', 'using', 'qualitative', 'and', 'quantitative', 'aggregation', '/', 'summarization', 'and', 'build', 'tools', '(', 'static', 'reports', ',', 'interactive', 'dashboards', ',', 'web', 'applications', ')', 'that', 'a

['To', 'begin', 'areas', 'of', 'focus', 'will', 'be', 'agency', 'MBS', ',', 'help', 'automate', 'various', 'analysis', 'using', 'python', ',', 'doing', 'ad', '-', 'hoc', 'prepayment', 'analysis', 'for', 'clients', ',', 'and', 'doing', 'research', 'of', 'combining', 'different', 'machine', 'learning', 'techniques', 'for', 'prepayment', 'modelling', '.']
[['python'], ['research']]
python 15 B |
research 28 B |
['The', 'individual', 'will', 'need', 'to', 'demonstrate', 'a', 'record', 'of', 'being', 'a', 'team', 'player', 'with', 'potential', 'to', 'partner', 'with', 'teams', 'of', 'diverse', 'functions', ':', 'technology', 'partners', ',', 'control', 'groups', ',', 'strategists', '/', 'modellers', ',', 'and', 'trading', '&', 'sales']
[['sales']]
sales 36 B |
['An', 'advanced', 'degree', 'in', 'physics', ',', 'engineering', ',', 'math', ',', 'statistics', ',', 'computer', 'science', 'or', 'other', 'quantitative', 'fields']
[['engineering'], ['math'], ['statistics'], ['computer', 'science']

industrial 28 B |engineering 29 I |
computer 31 B |science 32 I |
economics 39 B |
['The', 'team', 'conducts', 'a', 'variety', 'of', 'research', 'studies', 'and', 'supports', 'data', 'analytics', 'efforts', 'for', 'clients', 'across', 'the', 'Army', ',', 'Office', 'of', 'the', 'Secretary', 'of', 'Defense', 'and', 'greater', 'DoD', ',', 'conducting', 'high', '-', 'quality', 'research', 'with', 'practical', 'real', '-', 'world', 'implications', '.']
[['Office']]
Office 19 B |
['2', 'years', "'", 'relevant', 'experience', 'plus', 'a', 'Bachelor', "'s", 'Degree', 'in', 'Epidemiology', ',', 'Statistics', ',', 'Biostatistics', ',', 'Applied', 'Mathematics', ',', 'Operations', 'Research', ',', 'or', 'other', 'relevant', 'field', 'is', 'required', '.']
[['Statistics'], ['Operations', 'Research']]
Statistics 13 B |
Operations 20 B |Research 21 I |
['Experience', 'with', 'basic', 'statistical', 'analyses', '(', 'e.g.', ',', 't', '-', 'tests', ',', 'ANOVA', ',', 'factor', 'analysis', ',', 'linear

['Team', 'members', 'will', 'work', 'collaboratively', 'with', 'the', 'Noblis', 'data', 'analytics', 'architects', 'and', 'data', 'scientists', 'to', 'design', 'and', 'implement', 'an', 'offline', 'and', 'near', 'real', 'time', 'analytics', 'capability', ',', 'visualization', 'and', 'a', 'near', 'real', '-', 'time', 'dash', 'boarding', 'and', 'visualization', 'capabilities', 'to', 'support', 'Federal', 'Agencies', 'in', 'strengthening', 'their', 'security', 'posture', 'against', 'fraudulent', 'web', 'transactions', '.']
[['visualization']]
visualization 27 B |
['This', 'position', 'is', 'for', 'a', 'Software', 'Developer', 'with', 'exposure', 'to', 'primary', 'to', 'SAS', ',', 'Extract', ',', 'Transform', 'and', 'Load', '(', 'ETL', ')', 'and', 'Hadoop', '.']
[['SAS'], ['Hadoop']]
SAS 12 B |
Hadoop 23 B |
['The', 'engineer', '/', 'developer', 'is', 'also', 'responsible', 'for', 'suggesting', 'other', 'technologies', ',', 'architectures', ',', 'and', 'products', 'that', 'improve', 'proce

In [34]:
total_sentence_list = []
total_index_list = []

In [35]:
for index in range( 0, len(total_list) ):
    try:
        tuplp_list = []

        txt = total_list[index][0]
        doc = nlp(txt)

        temp_sentence = [str(token) for token in doc]
#         print( word_list[index] )

        my_word_list = []
        for w in word_list[index]:
            my_word_list.append( [str(token) for token in nlp(str(w))] )


        total_sentence_list.append( temp_sentence )
        my_ind = 0
        for l in my_word_list:
            temp_Flag = True
            for t in l:
                my_ind = temp_sentence.index( str(t), my_ind )

                label = ''
                if temp_Flag:
                    label= 'B'
                    temp_Flag = False
                else:
                    label= 'I'
                tuplp_list.append( ( label, my_ind ) )

                my_ind += 1

        total_index_list.append( tuplp_list )
        
    except:
        print( 'Error--------------------------------------------------------------------' )

In [36]:
print( total_sentence_list[:10] )
print( total_index_list[:10] )

[['The', 'Enterprise', 'Data', 'Scientist', 'I', 'will', 'be', 'a', 'valuable', 'part', 'of', 'the', 'Data', 'Science', 'team', 'who', 'provide', 'statistical', 'analysis', ',', 'modeling', ',', 'and', 'vis', 'ualization', 'services', 'for', 'all', 'lines', 'of', 'business', 'and', 'all', 'shared', 'service', 'functions', 'at', 'Farmers', 'Insurance', 'Group', '.'], ['Extract', 'data', 'from', 'a', 'variety', 'of', 'sources', '(', 'databases', ',', 'web', ',', 'text', 'files', ')', 'and', 'in', 'a', 'variety', 'of', 'formats', '(', 'structured', ',', 'unstructured', ',', 'JSON', ',', 'Parquet', ',', 'etc', '.', ')', 'and', 'prepare', 'the', 'data', 'for', 'analysis', 'Describe', 'data', 'using', 'qualitative', 'and', 'quantitative', 'aggregation', '/', 'summarization', 'and', 'build', 'tools', '(', 'static', 'reports', ',', 'interactive', 'dashboards', ',', 'web', 'applications', ')', 'that', 'allow', 'clients', 'to', 'easily', 'view', 'and', 'understand', 'important', 'patterns', 'and

In [37]:
total_result_list = []

In [38]:
for ind in range( 0, len(total_sentence_list) ):
#     print( total_sentence_list[ind] )
#     print( total_index_list[ind] )
    
    length = len( total_sentence_list[ind] )
    result_list = ['O'] * length
    
    for tem in total_index_list[ind]:
#         print( tem )
        result_list[tem[1]] = tem[0]
        
#     print( result_list )
    total_result_list.append( result_list )
    
#     print( '-' *120 )

In [39]:
import pickle

with open('total_sentence_list','wb') as out_data:
    pickle.dump(total_sentence_list,out_data)
    
with open('total_index_list','wb') as out_data:
    pickle.dump(total_index_list,out_data)

# sample dataset

In [40]:
total_result_list = []

In [41]:
for ind in range( 0, 100 ):
    print( total_sentence_list[ind] )
    print( total_index_list[ind] )
    
    length = len( total_sentence_list[ind] )
    result_list = ['O'] * length
    
    for tem in total_index_list[ind]:
#         print( tem )
        result_list[tem[1]] = tem[0]
        
    print( result_list )
    total_result_list.append( result_list )
    
    print( '-' *120 )

['The', 'Enterprise', 'Data', 'Scientist', 'I', 'will', 'be', 'a', 'valuable', 'part', 'of', 'the', 'Data', 'Science', 'team', 'who', 'provide', 'statistical', 'analysis', ',', 'modeling', ',', 'and', 'vis', 'ualization', 'services', 'for', 'all', 'lines', 'of', 'business', 'and', 'all', 'shared', 'service', 'functions', 'at', 'Farmers', 'Insurance', 'Group', '.']
[('B', 17), ('I', 18), ('B', 20)]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
------------------------------------------------------------------------------------------------------------------------
['Extract', 'data', 'from', 'a', 'variety', 'of', 'sources', '(', 'databases', ',', 'web', ',', 'text', 'files', ')', 'and', 'in', 'a', 'variety', 'of', 'formats', '(', 'structured', ',', 'unstructured', ',', 'JSON', ',', 'Parquet', ',', 'etc', '.', ')', 'and', 'prepare',